In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
from pycocotools.coco import COCO
import numpy as np
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (10.0, 8.0)
from torch.utils.data import DataLoader
import torch
from torch.autograd import Variable
from faster_rcnn.utils.cython_bbox import bbox_overlaps
from pycrayon import CrayonClient

import cPickle
from torch.optim import SGD, RMSprop, Adam
from torch.optim.lr_scheduler import StepLR
from datetime import datetime
from faster_rcnn.utils.datasets.adapter import convert_data
from faster_rcnn.utils.evaluate.metter import AverageMeter
from faster_rcnn.utils.display.images import imshow, result_show

In [3]:
with open('sorted_index', 'rb') as fp:
    sorted_index = cPickle.load(fp)

### Đọc dữ liệu từ MS COCO dataset


In [4]:
from faster_rcnn.utils.datasets.adapter import convert_data

In [5]:
import os
import torchvision.transforms as transforms
from faster_rcnn.utils.datasets.mscoco.dataset import CocoData
from torch.utils.data import DataLoader



dataDir = './data/mscoco'
dataType = 'train2014'
annFile='%s/annotations/instances_%s.json'%(dataDir,dataType)
batch_size = 4

images_dir = os.path.join(dataDir,'images', dataType)
cap = CocoData(root = images_dir, annFile = annFile,sorted_indexes=sorted_index)

train_data_loader = DataLoader(cap, batch_size=batch_size, shuffle=False, collate_fn=convert_data, num_workers=8)

loading annotations into memory...
Done (t=14.40s)
creating index...
index created!


In [6]:
train_data_loader.batch_size
print len(train_data_loader)

20696


Thử hiển thị ảnh cùng các bounding boxes

In [7]:
from faster_rcnn.faster_rcnn import FastRCNN


### Tính toán feed-forward


Chúng ta sử dụng một ảnh có kích thước đầu vào là  `(width , height) = (600, 800)`

Input:
    - im_data : 
        kích thước : (batch_size, dim, witdh, height)
    - ground_boxes: 
        kích thước (n, 4)
        

In [8]:
categories = ['__background__'] + [x['name'] for x in cap.coco.loadCats(cap.coco.getCatIds())]

In [9]:
net = FastRCNN(categories, debug=False)
net.cuda()
net.train()

/opt/conda/envs/pytorch/lib/python2.7/site-packages/torch/nn/parallel/data_parallel.py:24: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 0 which
    has less than 75% of the memory or cores of GPU 1. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))


FastRCNN(
  (rpn): RPN(
    (features): DataParallel(
      (module): Sequential(
        (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace)
        (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): ReLU(inplace)
        (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (6): ReLU(inplace)
        (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (8): ReLU(inplace)
        (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (11): ReLU(inplace)
        (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (13): ReLU(inplace)
        (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (15): 

In [10]:
param = filter(lambda x: x.requires_grad, net.parameters())
optimizer = SGD(param, lr=1e-3, momentum=0.9, weight_decay=0.0005)
exp_lr_scheduler = StepLR(optimizer, step_size=1000, gamma=0.95)

In [12]:
for i, blobs in enumerate(train_data_loader):
    if i > 10:
        break
    batch_tensor, im_info, batch_boxes, batch_boxes_index = blobs
    cls_prob, bbox_pred, rois, cls_score, target, rpn_cls_prob_reshape, rpn_bbox_pred, rpn_target = net(batch_tensor, im_info, batch_boxes, batch_boxes_index)
    rpn_cross_entropy, rpn_bbox_loss = net.rpn.build_loss(rpn_cls_prob_reshape, rpn_bbox_pred, rpn_target)
    cross_entropy, bbox_loss = net.build_loss(cls_score, bbox_pred, target)
    print(rpn_cross_entropy, rpn_bbox_loss)
    print(cross_entropy, bbox_loss)

No handlers could be found for logger "faster_rcnn.utils.datasets.mscoco.dataset"


(tensor(0.6985, device='cuda:0', grad_fn=<NllLossBackward>), tensor(0.3918, device='cuda:0', grad_fn=<MeanBackward1>))
(tensor(5.5202, device='cuda:0', grad_fn=<NllLossBackward>), tensor(0.5689, device='cuda:0', grad_fn=<MeanBackward1>))
(tensor(0.6927, device='cuda:0', grad_fn=<NllLossBackward>), tensor(0.5302, device='cuda:0', grad_fn=<MeanBackward1>))
(tensor(5.4387, device='cuda:0', grad_fn=<NllLossBackward>), tensor(0.2124, device='cuda:0', grad_fn=<MeanBackward1>))
(tensor(0.6938, device='cuda:0', grad_fn=<NllLossBackward>), tensor(0.6935, device='cuda:0', grad_fn=<MeanBackward1>))
(tensor(5.4229, device='cuda:0', grad_fn=<NllLossBackward>), tensor(0.2228, device='cuda:0', grad_fn=<MeanBackward1>))
(tensor(0.6949, device='cuda:0', grad_fn=<NllLossBackward>), tensor(0.3631, device='cuda:0', grad_fn=<MeanBackward1>))
(tensor(5.4094, device='cuda:0', grad_fn=<NllLossBackward>), tensor(0.1438, device='cuda:0', grad_fn=<MeanBackward1>))
(tensor(0.6932, device='cuda:0', grad_fn=<NllLos

In [ ]:
def train(data_gen ,model, tensorboard_client, metters, optimizer, lr_scheduler, current_epoch=0):
    
    steps_per_epoch = len(train_data_loader) 
    model.train()
    train_loss , cross_entropy , loss_box, rpn_loss = metters
    for step, blobs in enumerate(train_data_loader):
        if step < 20670:
            if step % 100 == 1:
                print(step, datetime.now().strftime('%m/%d_%H:%M'))
            continue
        batch_tensor, im_info, batch_boxes, batch_boxes_index = blobs
        print batch_tensor.shape
        optimizer.zero_grad()
        model(batch_tensor, im_info, batch_boxes, batch_boxes_index)
        cross_entropy.update(model.cross_entropy.item())
        loss_box.update(model.loss_box.item())
        train_loss.update(model.loss.item())
        rpn_loss.update(model.rpn.loss.item())
        loss = model.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()        


        current_step = current_epoch * steps_per_epoch + step
        if step % 30 == 1:
            log_text = 'epoch: %d : step %d,  loss: %.4f at %s' % (
                current_epoch + 1, step , train_loss.avg, datetime.now().strftime('%m/%d_%H:%M'))
            print(log_text)

        if step % 100 == 0:
            tensorboard_client.add_scalar_value('train_loss', train_loss.avg, step=current_step)
            tensorboard_client.add_scalar_value('rpn_loss', rpn_loss.avg, step=current_step)
            tensorboard_client.add_scalar_value('cross_entropy', cross_entropy.avg, step=current_step)
            tensorboard_client.add_scalar_value('loss_box', loss_box.avg, step=current_step)

In [15]:
def training(train_data_gen, optimizer, lr_scheduler ,model, epochs=10):
    
    exp_name = datetime.now().strftime('vgg16_%m-%d_%H-%Ms')
    cc = CrayonClient(hostname="crayon", port=8889)
    exp = cc.create_experiment(exp_name)
    
    
    train_loss = AverageMeter()
    cross_entropy = AverageMeter()
    loss_box = AverageMeter()
    rpn_loss = AverageMeter()
    metters = (train_loss , cross_entropy , loss_box, rpn_loss)

    for epoch in range(epochs):
        train(train_data_gen ,model, exp, metters, optimizer, lr_scheduler, epoch)

In [16]:
training(train_data_loader ,optimizer=optimizer,lr_scheduler=exp_lr_scheduler, model=net, epochs=10)

No handlers could be found for logger "faster_rcnn.utils.datasets.mscoco.dataset"


(1, '09/11_06:10')


No handlers could be found for logger "faster_rcnn.utils.datasets.mscoco.dataset"


(101, '09/11_06:10')


No handlers could be found for logger "faster_rcnn.utils.datasets.mscoco.dataset"


...

No handlers could be found for logger "faster_rcnn.utils.datasets.mscoco.dataset"
No handlers could be found for logger "faster_rcnn.utils.datasets.mscoco.dataset"


(601, '09/11_06:10')
(701, '09/11_06:10')
(801, '09/11_06:10')


No handlers could be found for logger "faster_rcnn.utils.datasets.mscoco.dataset"


(901, '09/11_06:10')
(1001, '09/11_06:10')
(1101, '09/11_06:11')
(1201, '09/11_06:11')
(1301, '09/11_06:11')
(1401, '09/11_06:11')
(1501, '09/11_06:11')
(1601, '09/11_06:11')
(1701, '09/11_06:11')
(1801, '09/11_06:11')
(1901, '09/11_06:11')
(2001, '09/11_06:11')
(2101, '09/11_06:11')
(2201, '09/11_06:11')
(2301, '09/11_06:11')
(2401, '09/11_06:11')
(2501, '09/11_06:12')
(2601, '09/11_06:12')
(2701, '09/11_06:12')
(2801, '09/11_06:12')
(2901, '09/11_06:12')
(3001, '09/11_06:12')
(3101, '09/11_06:12')
(3201, '09/11_06:12')
(3301, '09/11_06:12')
(3401, '09/11_06:12')
(3501, '09/11_06:12')
(3601, '09/11_06:12')
(3701, '09/11_06:12')
(3801, '09/11_06:12')
(3901, '09/11_06:12')
(4001, '09/11_06:13')
(4101, '09/11_06:13')
(4201, '09/11_06:13')
(4301, '09/11_06:13')
(4401, '09/11_06:13')
(4501, '09/11_06:13')
(4601, '09/11_06:13')
(4701, '09/11_06:13')
(4801, '09/11_06:13')
(4901, '09/11_06:13')
(5001, '09/11_06:13')
(5101, '09/11_06:13')
(5201, '09/11_06:13')
(5301, '09/11_06:13')
(5401, '09/

RuntimeError: cuda runtime error (2) : out of memory at /opt/conda/conda-bld/pytorch_1532571898140/work/aten/src/THC/generic/THCTensorMath.cu:35

In [ ]:
net.load_state_dict(torch.load('faster.pkl'))

In [ ]:
pred_boxes, scores, classes, rois, im_data = net.detect("./test7.jpg", thr=0.5)
print classes
print scores
print pred_boxes
imshow(im_data[0],[] ,pred_boxes, random=True)

In [ ]:
print 1

In [ ]:
__